In [73]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re


## scrping av del 1

In [54]:
from bs4 import BeautifulSoup, NavigableString
import requests

def extract_text(url):
    # Get the HTML of the page
    response = requests.get(url)
    html = response.text

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the divs with the specified classes
    divs = soup.find_all(class_=["RichtextArea ProductPage__richtext text-wrapper", "layout--2col wrapper"])

    # Extract the text of each child of each div until a h2, h3 or ul tag is encountered
    text = []
    for div in divs:
        for child in div.children:
            if isinstance(child, NavigableString):
                text.append(child.strip())
            elif child.name in ['ul']:
                break
            else:
                text.append(child.get_text(strip=True))
    text_string = ' '.join(text)
        
    return text_string
df = pd.read_csv("data/tags_30_11.csv")
number = 23

print(extract_text(df["url"][number]))

print(df["url"][number])

 EK80 Portable EntryThe EK80 Portable Entry is all you need for ecosystem monitoring and analysis in one package – at a reasonable price.  EK80 Portable Entry bundle comprises:   This is a complete rugged echo sounder with a built-in computer, GPS, and WiFi for remote operation. The core of the product is the WBT Mini, a compact EK80 wideband transceiver with four channels. The Entry bundle uses three of these channels for the 38 kHz split beam, while the last channel is used for the 200 kHz single beam.    The EK80 Portable entry is provided with the ES38-18/200-18 transducer. This is a dual-frequency transducer combining 38 and 200 kHz in one housing. The beamwidth is 18 degrees at both frequencies to secure a large sampling volume in shallow water. Combining high and low frequencies is ideal for mapping ecosystem components and separating fish from plankton.   Key features 
https://www.kongsberg.com/maritime/products/ocean-science/ocean-science/es_scientific/sci_es_transceivers/ek80

## scraping av data sheets

In [115]:
from bs4 import BeautifulSoup
import requests
import re

def find_datasheets(url):
    # Get the HTML of the page
    response = requests.get(url)
    html = response.text

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the div with the class "Downloads"
    downloads_div = soup.find('div', class_="Downloads")

    # If the div is found, find the section with the subtitle "Data sheet", "DATA SHEET", or "Data- and product sheets" within this div
    if downloads_div:
        datasheets_subtitle = downloads_div.find('h3', class_="Section__subtitle", string=re.compile("data sheet|data- and product sheets|Brochure", re.I))

        # If the subtitle is found, find the next sibling section
        if datasheets_subtitle:
            datasheets_section = datasheets_subtitle.find_next_sibling()

            # If the section is found, find all links within this section
            if datasheets_section:
                datasheet_links = datasheets_section.find_all('a', class_="Downloads__itemLink")

                # If the links are found, prepend "https://www.kongsberg.com" to each href attribute and return the list
                if datasheet_links:
                    return ["https://www.kongsberg.com" + link.get('href') for link in datasheet_links if link.get('href').endswith('.pdf')]
        else:
            # If no subtitle is found, find all links where the direct child span has the text "Datasheet"
            datasheet_links = downloads_div.find_all('a', class_="Downloads__itemLink")
    
            # If the links are found, return the href attribute of the first link that ends with ".pdf"
            for link in datasheet_links:
                href = link.get('href')
                if href.endswith('.pdf'):
                    return ["https://www.kongsberg.com" + href]

    # If the section, the div, or the links are not found, return None
    return None

df = pd.read_csv("data/tags_30_11.csv")
number = 83

print(find_datasheets(df["url"][number]))
print(df["url"][number])

['https://www.kongsberg.com/globalassets/discovery/simrad/transducers/200-and-es200/164011_200-7g.pdf']
https://www.kongsberg.com/maritime/products/commercial-fisheries/td/200-khz/simrad-200-7g/


In [86]:
df = pd.read_csv("data/tags_30_11.csv")

# Add a new column "Data sheets" to the dataframe
df['Data sheets'] = df['url'].apply(find_datasheets)



In [87]:
# only keep column url, product name and data sheets
df = df[['url', 'Product_Name', 'Data sheets']]
df.to_csv("data/data_sheets.csv", index=False)
df.to_excel("data/data_sheets.xlsx", index=False)